# Compare fit to square foot calculation

How much did the prototype buildings approach really improve things relative to just using lot area?

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('asu-light')

In [ ]:
# Get the buildable areas and existing feasibility
data = pd.read_sql("""
SELECT f.*, ST_Area(ba.geog)
FROM diss.building_fit f
LEFT JOIN diss.buildable_areas ba USING (gid)
LEFT JOIN diss.gp16 p USING (gid)
 WHERE p.scag_zn_co IN (
            '1110', -- single family residential
            '1111', -- high dens SF residential
            '1112', -- med dens SF residential
            '1113', -- low dens SF residential
            '1150'  -- rural residential
        )
        AND p.Building_PropertyLandUseStndCode IN ('RR101', 'VL101')
        AND (total_rent IS NOT NULL OR vacant_npv IS NOT NULL)
        AND p.puma IS NOT NULL
""", "postgresql://matthewc@localhost/matthewc")

In [ ]:
data['simple_fit_sfh_duplex'] = data.st_area >= (12 * 10)
data['simple_fit_threeplex'] = data.st_area >= (16 * 11)
data['simple_fit_sixplex'] = data.st_area >= (21 * 10)

In [ ]:
# make sure that the simple fit is (almost) an upper bound
assert (~data.loc[data.fit_sfh_duplex, 'simple_fit_sfh_duplex']).sum() <= 100
assert (~data.loc[data.fit_fourplex, 'simple_fit_threeplex']).sum() <= 500  # a few right on the edge of feasibility have some rasterization error
assert (~data.loc[data.fit_sixplex, 'simple_fit_sixplex']).sum() <= 500  # same

In [ ]:
(~data.loc[data.fit_sfh_duplex, 'simple_fit_sfh_duplex']).sum()

In [ ]:
(~data.loc[data.fit_sixplex, 'simple_fit_sixplex']).sum()

In [ ]:
data.loc[data.fit_sixplex & ~data.simple_fit_sixplex, ['gid', 'st_area']]

Originally 0.13929388180671326, data used here is corrected (see note on page 34 of dissertation).

In [ ]:
(
    (data.simple_fit_sfh_duplex & ~data.fit_sfh_duplex) |
    (data.simple_fit_threeplex & ~data.fit_fourplex) |
    (data.simple_fit_sixplex & ~data.fit_sixplex)
).mean()

In [ ]:
wrd = data.loc[(
    (~data.simple_fit_sfh_duplex & data.fit_sfh_duplex) |
    (~data.simple_fit_threeplex & data.fit_fourplex) |
    (~data.simple_fit_sixplex & data.fit_sixplex)
), 'gid'].sample(100)

In [ ]:
wrdg = gp.read_postgis("SELECT gid, ST_Transform(geog::geometry, 26911) AS geom FROM diss.gp16 WHERE gid IN ({})".format(','.join(map(str, wrd))), "postgresql://matthewc@localhost/matthewc")

In [ ]:
wrdg.geom.iloc[3]

In [ ]:
wrdg.iloc[3]

In [ ]:
wrdg.iloc[[34]].plot()
plt.xticks([])
plt.yticks([])

In [ ]:
wrdg.iloc[[34]].to_crs(epsg=4326)

In [ ]:
data[data.gid == 921834]